In [1]:
from utils import *
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import poisson, chisquare
import pickle

In [2]:
with open('params.pickle', 'rb') as f:
    saved_params = pickle.load(f)

params = saved_params[0]
team_params = saved_params[1]
streak_param = saved_params[2]

In [3]:
params.shape

(4, 20)

In [4]:
zero = np.zeros((3,3))
zero[1,1]=1
zero[1,2]=2
np.triu(zero,1)

array([[0., 0., 0.],
       [0., 0., 2.],
       [0., 0., 0.]])

In [12]:
prob = get_probabilities(params, 0, 1)


In [6]:
prob

{'H': 0.7983712830379134, 'D': 0.14369785589433204, 'A': 0.0579308554114443}

In [7]:
sum(prob.values())

0.9999999943436897